In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import spacy
import random
import logging
import srsly
from spacy.gold import docs_to_json, biluo_tags_from_offsets, spans_from_biluo_tags
import os
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
from tqdm import tqdm

In [ ]:
# change directory to train folder
cd /content/drive/MyDrive/MER/MER train

/content/drive/.shortcut-targets-by-id/1moJYPeCIU-ElBuY9ZusCH9hileAHoqge/train_data


In [ ]:
ls .

ADR_1.txt    train12.txt  train17.txt  train3.txt  train7.txt
ADR_2.txt    train13.txt  train18.txt  train4.txt  train8.txt
train10.txt  train14.txt  train19.txt  train5.txt  train9.txt
train11.txt  train16.txt  train2.txt   train6.txt  train.txt


In [ ]:
# annotated text files for training
train_data = ['./train13.txt','./train17.txt','./train2.txt','./train5.txt','./train8.txt', './train10.txt', \
              './train11.txt','./train14.txt','./train18.txt','./train3.txt','./train6.txt','./train9.txt',\
              './train12.txt','./train16.txt','./train19.txt','./train4.txt','./train7.txt', './ADR_1.txt', './ADR_2.txt']

         

In [ ]:
# writing the annotated train text files to a single text file
with open('/content/drive/MyDrive/medical text/train_data/train.txt', 'w') as outfile:
  for fname in train_data:
    with open(fname) as infile:
      for line in infile:
        outfile.write(line)

In [ ]:
# loading the train data
import ast
TRAIN = [] 
for fname in train_data:
    f = open(fname,"r")
    d = (f.read())
    y = ast.literal_eval(d)
    TRAIN.extend(y)


In [ ]:
# loading the test data
import ast
TEST = [] 
for fname in test_data:
    f = open(fname,"r")
    d = (f.read())
    y = ast.literal_eval(d)
    TEST.extend(y)


In [ ]:
from pprint import pprint 
pprint (Train_data)
#print(type(Train_data))

Streaming output truncated to the last 5000 lines.
 ('Could this be the cause?', {'entities': []}),
 ('Troynow, are you talking about development of facial hair or voice problem.',
  {'entities': [(53, 57, 'BODY PART')]}),
 ('Or is it just that you look young for your age.', {'entities': []}),
 ('I am happy to say I look young for my age and glad about it!!',
  {'entities': []}),
 ('Kehnie, need to know what sort of discharge you are getting from your '
  'breast and its relationship to your cycle.',
  {'entities': [(34, 43, 'SYMPTOM'), (70, 76, 'BODY PART')]}),
 ('How old you are also makes a difference doc it is nt visible unless my '
  'boyfriend suck it,am 25 yrs.',
  {'entities': [(40, 43, 'PERSON'), (71, 80, 'PERSON')]}),
 ('Yes', {'entities': []}),
 ('Dr Balogun', {'entities': [(0, 2, 'PERSON'), (3, 10, 'PERSON')]}),
 (', i look young for my age.', {'entities': []}),
 ('My face shows this, and im not happy about it.',
  {'entities': [(3, 7, 'BODY PART')]}),
 ('Is there anything 

In [ ]:
nlp = spacy.load("en") # load spacy English model

In [ ]:
# convert the annotated train file to spacy gold format in JSON
docs = []
for text, annot in TRAIN:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot['entities'])
    entities = spans_from_biluo_tags(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json("spacy_format1.json", [docs_to_json(docs)])

In [ ]:
# convert the annotated test file to spacy gold format in JSON
docs = []
for text, annot in TEST:
    doc = nlp(text)
    tags = biluo_tags_from_offsets(doc, annot['entities'])
    entities = spans_from_biluo_tags(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json("spacy_format2.json", [docs_to_json(docs)])

In [ ]:
!python -m spacy debug-data en 'spacy_format1.json' 'spacy_format2.json'


=========================== Data format validation ===========================
✔ Corpus is loadable

=============================== Training stats ===============================
Training pipeline: tagger, parser, ner
Starting with blank model 'en'
9513 training docs
1057 evaluation docs
⚠ 22 training examples also in evaluation data

============================== Vocab & Vectors ==============================
ℹ 139908 total words in the data (10138 unique)
ℹ No word vectors present in the model

========================== Named Entity Recognition ==========================
ℹ 15 new labels, 0 existing labels
0 missing values (tokens with '-' label)
⚠ 78 entity span(s) with punctuation
⚠ Low number of examples for new label 'DOSAGE' (16)
⚠ Low number of examples for new label 'PHISIOLOGIC PROCESS' (17)
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with whitespace
Entity spans consisting of or starting/ending with punctuation ca

In [ ]:
cd /content/drive/MyDrive/MER

/content/drive/MyDrive/MER


In [ ]:
# first training
!python -m spacy train en spacy_model_for_mer 'spacy_format1.json' 'spacy_format2.json'

✔ Created output directory: spacy_model_for_mer
Training pipeline: ['tagger', 'parser', 'ner']
Starting with blank model 'en'
Counting training words (limit=0)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W022] Training a new part-of-speech tagger using a model with no lemmatization rules or data. This means that the trained model may not be able to lemmatize correctly. If this is intentional or the language you're using doesn't have lemmatization data, you can ignore this warning by setting SPACY_WARNING_IGNORE=W022. If this is surprising, make sure you have the spacy-lookups-data package installed.
  "__main__", mod_spec)

Itn  Tag Loss    Tag %    Dep Loss    UAS     LAS    NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  --------  ---------  ------  ------  ---------  ------  ------  ------  -------  -------
  1  28377.886    93.714  140496.482  82.349  76.909  13079.445  67.545  49.001  56.798   99.818     6880
  2  12517.016    94.692  94573.629  84.967  80.7

In [ ]:
# second training
!python -m spacy train en latest_spacy_model_for_mer 'spacy_train_format.json' 'spacy_format2.json'

✔ Created output directory: latest_spacy_model_for_mer
Training pipeline: ['tagger', 'parser', 'ner']
Starting with blank model 'en'
Counting training words (limit=0)
/usr/lib/python3.6/runpy.py:193: UserWarning: [W022] Training a new part-of-speech tagger using a model with no lemmatization rules or data. This means that the trained model may not be able to lemmatize correctly. If this is intentional or the language you're using doesn't have lemmatization data, you can ignore this warning by setting SPACY_WARNING_IGNORE=W022. If this is surprising, make sure you have the spacy-lookups-data package installed.
  "__main__", mod_spec)

Itn  Tag Loss    Tag %    Dep Loss    UAS     LAS    NER Loss   NER P   NER R   NER F   Token %  CPU WPS
---  ---------  --------  ---------  ------  ------  ---------  ------  ------  ------  -------  -------
  1  29354.835    91.681  131877.690  77.323  42.370  13161.976  66.031  45.091  53.588   99.818     6743
  2  13111.305    92.294  85765.840  79.88

### Testing the model

In [ ]:
mer_model = spacy.load("/content/drive/MyDrive/MER/latest_spacy_model_for_mer/model-best")

In [ ]:
sentence = "Whenever I hear SEPTRIN, my heart aches. This drug enlarged my breasts, five minutes after application,\
 with watery discharge from the breasts."

doc = mer_model(sentence)
spacy.displacy.render(doc, style='ent', jupyter=True)

In [ ]:
sentence = "My private is itching, stooling is painful\
              Doc gave coartem and antibiotic to be taken 2 tablets for 5 days"

doc = mer_model(sentence)
spacy.displacy.render(doc, style='ent', jupyter=True)

In [ ]:
sentence = "I took alabukun powder, i started having pains in my lower abdomen \
 and started bleeding, i decided to go for checkup in the hospital"

doc = mer_model(sentence)
spacy.displacy.render(doc, style='ent', jupyter=True)

In [ ]:
sentence = "Mine was very bad it landed me in the hospital. I was admitted for three days on drip.\
i took Fansidar and the next day my face was swollen,it affected my throat,then the fever grew worse"

doc = mer_model(sentence)
spacy.displacy.render(doc, style='ent', jupyter=True)

In [ ]:
# sentence = "I had a friend who reacted to septrin. Was very bad. \
# Itching all over with some part of skin getting dark"

# doc = mer_model(sentence)
# spacy.displacy.render(doc, style='ent', jupyter=True)

*____________________________________________________________________*